## 10.1 Panel Data

- In contrast to cross-section data where we have observations on n subjects (entities), panel data has observations on n entities at T ≥ 2 time periods.
- Where the index i refers to the entity while t refers to the time period, this is denoted

\begin{equation}
(X_{it},Y_{it}), \ i=1,\dots,n \ \ \ \text{and} \ \ \ t=1,\dots,T
\end{equation}

- Sometimes panel data is also called longitudinal data as it adds a temporal dimension to cross-sectional data. 
- Let us have a look at the dataset Fatalities by checking its structure and listing the first few observations.

In [3]:
using CSV #we use the CSV package to load the data
using DataFrames #we use the DataFrames package as the data is stored as an object of type "DataFrame"

fatalities = CSV.read("/mnt/juliabox/Econometrics With Julia/Datasets/fatalities.csv")

,Column1,state,year,spirits,unemp,income,emppop,beertax,baptist,mormon,drinkage,dry,youngdrivers,miles,breath,jail,service,fatal,nfatal,sfatal,fatal1517,nfatal1517,fatal1820,nfatal1820,fatal2124,nfatal2124,afatal,pop,pop1517,pop1820,pop2124,milestot,unempus,emppopus,gsp
,Int64⍰,String⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,String⍰,String⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,1,al,1982,1.37,14.4,10544.2,50.692,1.53938,30.3557,0.32829,19.0,25.0063,0.211572,7233.89,no,no,no,839,146,99,53,9,99,34,120,32,309.438,3.942e6,2.09e5,2.21553e5,2.9e5,28516.0,9.7,57.8,-0.0221248
2,2,al,1983,1.36,13.7,10732.8,52.147,1.78899,30.3336,0.34341,19.0,22.9942,0.210768,7836.35,no,no,no,930,154,98,71,8,108,26,124,35,341.834,3.96001e6,2.02e5,2.19125e5,2.9e5,31032.0,9.6,57.9,0.0465583
3,3,al,1984,1.32,11.1,11108.8,54.1681,1.71429,30.3115,0.35924,19.0,24.0426,0.211484,8262.99,no,no,no,932,165,94,49,7,103,25,118,34,304.872,3.98899e6,197000.0,2.16724e5,2.88e5,32961.0,7.5,59.5,0.0627978
4,4,al,1985,1.28,8.9,11332.6,55.2711,1.65254,30.2895,0.37579,19.67,23.6339,0.21114,8726.92,no,no,no,882,146,98,66,9,100,23,114,45,276.742,4.02101e6,1.95e5,214349.0,2.84e5,35091.0,7.2,60.1,0.02749
5,5,al,1986,1.23,9.8,11661.5,56.5145,1.60991,30.2674,0.39311,21.0,23.4647,0.2134,8952.85,no,no,no,1081,172,119,82,10,120,23,119,29,360.716,4.04999e6,2.04e5,212000.0,2.63e5,36259.0,7.0,60.7,0.0321429
6,6,al,1987,1.18,7.8,11944.0,57.5099,1.56,30.2453,0.41123,21.0,23.7924,0.215527,9166.3,no,no,no,1110,181,114,94,11,127,31,138,30,368.421,4.083e6,2.05e5,2.08998e5,2.59e5,37426.0,6.2,61.5,0.0489764
7,7,al,1988,1.17,7.2,12368.6,56.8345,1.50144,30.2233,0.43018,21.0,23.7924,0.218328,9674.32,no,no,no,1023,139,89,66,8,105,24,123,25,298.322,4.10199e6,2.01e5,1.93001e5,2.63e5,39684.0,5.5,62.3,0.0353918
8,8,az,1982,1.97,9.9,12309.1,56.8933,0.214797,3.9589,4.9191,19.0,0.0,0.209012,6810.16,no,yes,yes,724,131,76,40,7,81,16,96,36,173.668,2.897e6,141000.0,1.56379e5,218000.0,19729.0,9.7,57.8,-0.0431819
9,9,az,1983,1.9,9.1,12693.8,57.5536,0.206422,3.8901,4.83107,19.0,0.0,0.203855,6587.49,no,yes,yes,675,112,60,40,7,83,19,80,17,196.89,2.977e6,1.39e5,1.57521e5,2.19e5,19611.0,9.6,57.9,0.0762055


- We find that the dataset consists of 336 observations on 34 variables. 
- Notice that the variable state is a factor variable with 48 levels (one for each of the 48 contiguous federal states of the U.S.). 
- The variable year is also a factor variable that has 7 levels identifying the time period when the observation was made. 
- This gives us 7 × 48 = 336 observations in total. 
- Since all variables are observed for all entities and over all time periods, the panel is balanced. 
- If there were missing data for at least one entity in at least one time period we would call the panel unbalanced.

### Example: Traffic Deaths and Alcohol Taxes

- We start by reproducing Figure 10.1 of the book. 
- To this end we estimate simple regressions using data for years 1982 and 1988 that model the relationship between beer tax (adjusted for 1988 dollars) and the traffic fatality rate, measured as the number of fatalities per 10000 inhabitants. 
- Afterwards, we plot the data and add the corresponding estimated regression functions.
- If the kernel dies, try running the code using a local Julia environment.

In [ ]:
using FixedEffects #FixedEffectModels requires FixedEffects as a dependency
using FixedEffectModels #we use FixedEffectModels to create regression models
using CSV #we use the CSV package to load the data
using DataFrames #we use the DataFrames package as the data is stored as an object of type "DataFrame"
using Query #we use the Query package in order to filter the data using LINQ style query commands
using Plots 

fatalities = CSV.read("/mnt/juliabox/Econometrics With Julia/Datasets/fatalities.csv")

fatalities1982 = @from i in fatalities begin
    @where i.year == 1982
    @select {i.beertax, fatality_rate = i.fatal / i.pop * 10000}
    @collect DataFrame
end

fatalities1988 = @from i in fatalities begin
    @where i.year == 1988
    @select {i.beertax, fatality_rate = i.fatal / i.pop * 10000}
    @collect DataFrame
end

fatalities1982_model = reg( #initialise a FixedEffectModel and define it as fatalities1982_model
                        fatalities1982, #pass the DataFrame 'fatalities1982' as the dataset to be used in fatalities1982_model
                        @model(fatality_rate ~ beertax), #pass the regression formula consisting of the dependent variable 'fatality_rate' and the exogenous variable 'beertax'
)

fatalities1988_model = reg( #initialise a FixedEffectModel and define it as fatalities1988_model
                        fatalities1988, #pass the DataFrame 'fatalities1988' as the dataset to be used in fatalities1988_model
                        @model(fatality_rate ~ beertax), #pass the regression formula consisting of the dependent variable 'fatality_rate' and the exogenous variable 'beertax'
)

println(fatalities1982_model)
println(fatalities1988_model)



- The estimated regression functions are
\begin{align*}
  \widehat{FatalityRate} =& \, \underset{(0.15)}{2.01} + \underset{(0.13)}{0.15} \times BeerTax \quad (1982 \text{ data}), \\
  \widehat{FatalityRate} =& \, \underset{(0.11)}{1.86} + \underset{(0.13)}{0.44} \times BeerTax \quad (1988 \text{ data}).
\end{align*}

In [ ]:
x = fatalities1982.beertax
y = fatalities1982.fatality_rate

p1 = plot( #assign a plot object to the variable p1 using the following attributes
    x, #x series
    y, #y series
    st = :scatter, #series type
    title = "Traffic Fatality Rates and Beer Taxes in 1982", #plot title
    label = "US State", #legend labels
    xlabel = "Beer tax (in 1988 dollars)", #x axis label
    ylabel = "Fatality rate (fatalities per 10000)", #y axis label
    ylims = (0,4.5), #y axis limits
    yticks = 0:1:4.5, #y axis tick range
    ms = 4, #marker size
    mc = :blue #marker color
)

y_prediction_1982(xdata) = dot(coef(fatalities1982_model), [1, xdata]) #create a function that takes the dot product of the coefficients of the 'fatalities1982_model', 1 and the given x value 

x = [minimum(fatalities1982.beertax), maximum(fatalities1982.beertax)]
y = y_prediction_1982.(x) #broadcast the y function over the range of x values

plot!(
    p1,
    x,
    y,
    st = :line,
    label = "OLS"
)

display(p1)

x = fatalities1988.beertax
y = fatalities1988.fatality_rate

p2 = plot( #assign a plot object to the variable p2 using the following attributes
    x, #x series
    y, #y series
    st = :scatter, #series type
    title = "Traffic Fatality Rates and Beer Taxes in 1988", #plot title
    label = "US State", #legend labels
    xlabel = "Beer tax (in 1988 dollars)", #x axis label
    ylabel = "Fatality rate (fatalities per 10000)", #y axis label
    ylims = (0,4.5), #y axis limits
    yticks = 0:1:4.5, #y axis tick range
    ms = 4, #marker size
    mc = :blue #marker color
)

y_prediction_1988(xdata) = dot(coef(fatalities1988_model), [1, xdata]) #create a function that takes the dot product of the coefficients of the 'fatalities1988_model', 1 and the given x value

x = [minimum(fatalities1988.beertax), maximum(fatalities1988.beertax)] 
y = y_prediction_1988.(x)  #broadcast the y function over the range of x values

plot!(
    p2,
    x,
    y,
    st = :line,
    label = "OLS"
)

display(p2)
